In [37]:
# 0) Paths
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path("..").resolve()
RES_DIR = PROJECT_ROOT / "results"
RES_DIR.mkdir(exist_ok=True, parents=True)

# File Paths
pd_path   = RES_DIR / "stress_pd_summary.csv"      # scenario, family, mean_pd, p50, p90, p99, uplift_vs_baseline_pct
el_path   = RES_DIR / "stress_el_summary.csv"      # scenario, family, mean_pd, EL, EL_change_vs_baseline_pct
macro_all_path = RES_DIR / "macro_scenarios_all.csv"  # data-driven + Fed scenarios (macro)
genai_path     = RES_DIR / "genai_stress_scenarios.csv"      # GenAI macro scenarios

In [38]:
# 1) Load risk tables

pd_df = pd.read_csv(pd_path)
el_df = pd.read_csv(el_path)
macro_all_df = pd.read_csv(macro_all_path)
genai_df = pd.read_csv(genai_path)

# Clean colnames (strip whitespace)
pd_df.columns = [c.strip() for c in pd_df.columns]
el_df.columns = [c.strip() for c in el_df.columns]

print("PD DF columns:", pd_df.columns.tolist())
print("EL DF columns:", el_df.columns.tolist())
print("Macro all DF columns:", macro_all_df.columns.tolist())
print("GenAI DF columns:", genai_df.columns.tolist())


PD DF columns: ['scenario', 'family', 'mean_pd', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct']
EL DF columns: ['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct']
Macro all DF columns: ['scenario', 'family', 'GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'UNRATE_delta_qoq', 'FEDFUNDS_delta_qoq', 'GDPC1_delta_qoq', 'inflation_qoq', 'real_rate_qoq']
GenAI DF columns: ['scenario', 'GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'GDPC1_delta_qoq', 'UNRATE_delta_qoq', 'CPIAUCSL_delta_qoq', 'FEDFUNDS_delta_qoq', 'inflation_qoq', 'real_rate_qoq']


In [39]:
el_df = el_df.drop(columns=["family", "mean_pd"])

In [40]:
# Ensure required columns exist
required_pd = {"scenario", "family", "mean_pd"}
required_el = {"scenario", "EL", "EL_change_vs_baseline_pct"}

missing_pd = required_pd - set(pd_df.columns)
missing_el = required_el - set(el_df.columns)
if missing_pd:
    raise ValueError(f"PD summary is missing columns: {missing_pd}")
if missing_el:
    raise ValueError(f"EL summary is missing columns: {missing_el}")

# Merge PD + EL on (scenario, family, mean_pd)
risk_df = pd.merge(
    pd_df,
    el_df[list(required_el)],  # keep only key + EL info
    on= "scenario",
    how="left",
    validate="one_to_one"
)

print("Merged risk_df shape:", risk_df.shape)
display(risk_df.head())

Merged risk_df shape: (16, 9)


,scenario,family,mean_pd,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,EL_change_vs_baseline_pct,EL
0,baseline_actual,baseline,0.204853,0.175275,0.391406,0.617661,0.000000,0.000000,7.295394e+08
1,mild_adverse,data_driven,0.263438,0.253374,0.417717,0.566075,28.598151,28.598151,9.381742e+08
2,severe_adverse,data_driven,0.151683,0.139560,0.255326,0.384049,-25.955562,-25.955562,5.401834e+08
3,Fed_Adverse,Fed,0.022956,0.019596,0.040539,0.071352,-88.794167,-88.794167,8.175096e+07
4,Fed_Baseline,Fed,0.242373,0.231391,0.388901,0.536454,18.315129,18.315129,8.631555e+08


In [41]:
# 2) Load macro scenario tables (Fed + data-driven + GenAI)

macro_df_list = []

# --- Load FED + DATA-DRIVEN macro scenarios ---
if macro_all_path.exists():
    macro_all = pd.read_csv(macro_all_path)
    macro_all.columns = macro_all.columns.str.strip()

    # Keep only macro features + scenario
    fed_macro_cols = [
        "scenario", "GDPC1", "UNRATE", "CPIAUCSL", "FEDFUNDS",
        "UNRATE_delta_qoq", "FEDFUNDS_delta_qoq",
        "GDPC1_delta_qoq", "inflation_qoq", "real_rate_qoq"
    ]
    macro_all = macro_all[fed_macro_cols]

    macro_all["macro_source"] = "macro_all"
    macro_df_list.append(macro_all)

# --- Load GenAI macro scenarios ---
if genai_path.exists():
    genai_macro = pd.read_csv(genai_path)
    genai_macro.columns = genai_macro.columns.str.strip()

    # Some GenAI tables include CPIAUCSL_delta_qoq; keep if exists
    genai_keep = [c for c in [
        "scenario", "GDPC1", "UNRATE", "CPIAUCSL", "FEDFUNDS",
        "UNRATE_delta_qoq", "FEDFUNDS_delta_qoq",
        "GDPC1_delta_qoq", "inflation_qoq", "real_rate_qoq",
        "CPIAUCSL_delta_qoq"  # may or may not exist
    ] if c in genai_macro.columns]

    genai_macro = genai_macro[genai_keep]
    genai_macro["macro_source"] = "GenAI"
    macro_df_list.append(genai_macro)

# === Clean + merge ===
if macro_df_list:
    # Combine all macro tables
    macro_df = pd.concat(macro_df_list, ignore_index=True)

    if "scenario" not in macro_df.columns:
        raise ValueError("Macro scenario tables must contain a 'scenario' column.")

    # If scenario appears both in macro_all and genAI → keep macro_all version
    macro_df = (
        macro_df.sort_values("macro_source")
                .drop_duplicates(subset=["scenario"], keep="first")
                .reset_index(drop=True)
    )

    # Identify macro feature columns
    macro_cols = [
        c for c in macro_df.columns
        if c not in {"scenario", "macro_source"}
    ]

    print("Macro columns:", macro_cols)

    # Final merge risk + macro
    full_df = pd.merge(
        risk_df,                          # PD + EL
        macro_df[["scenario"] + macro_cols], 
        on="scenario",
        how="left",
        validate="one_to_one"
    )

else:
    macro_cols = []
    full_df = risk_df.copy()
    print("No macro scenario files found; JSON will have empty 'macro' blocks.")

print("Final merged full_df shape:", full_df.shape)
display(full_df.head())


Macro columns: ['GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'UNRATE_delta_qoq', 'FEDFUNDS_delta_qoq', 'GDPC1_delta_qoq', 'inflation_qoq', 'real_rate_qoq', 'CPIAUCSL_delta_qoq']
Final merged full_df shape: (16, 19)


,scenario,family,mean_pd,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,EL_change_vs_baseline_pct,EL,GDPC1,UNRATE,CPIAUCSL,FEDFUNDS,UNRATE_delta_qoq,FEDFUNDS_delta_qoq,GDPC1_delta_qoq,inflation_qoq,real_rate_qoq,CPIAUCSL_delta_qoq
0,baseline_actual,baseline,0.204853,0.175275,0.391406,0.617661,0.000000,0.000000,7.295394e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mild_adverse,data_driven,0.263438,0.253374,0.417717,0.566075,28.598151,28.598151,9.381742e+08,18782.243000,5.433333,247.238333,1.203333,-0.013072,0.202216,0.004002,0.006348,0.154176,NaN
2,severe_adverse,data_driven,0.151683,0.139560,0.255326,0.384049,-25.955562,-25.955562,5.401834e+08,17953.974000,6.933333,251.686333,1.923333,0.006803,0.555556,0.001845,0.007955,0.190550,NaN
3,Fed_Adverse,Fed,0.022956,0.019596,0.040539,0.071352,-88.794167,-88.794167,8.175096e+07,18821.441681,5.475000,257.030458,0.225000,0.800000,-0.500000,-0.035000,0.013000,-0.513000,NaN
4,Fed_Baseline,Fed,0.242373,0.231391,0.388901,0.536454,18.315129,18.315129,8.631555e+08,21214.070968,3.925000,260.107147,1.775000,0.000000,0.300000,0.028000,0.019000,0.281000,NaN


In [42]:

# 3) Build LLM-ready JSON

scenario_records = []

for _, row in full_df.iterrows():
    scen_name = str(row["scenario"])
    family    = str(row["family"])

    # ---- risk block ----
    risk_block = {
        "mean_pd": float(row["mean_pd"]),
    }
    # Optional PD distribution metrics if present
    for c in ["p50_pd", "p90_pd", "p99_pd", "uplift_vs_baseline_pct"]:
        if c in full_df.columns and pd.notna(row.get(c, None)):
            risk_block[c] = float(row[c])

    # EL info from EL summary
    if "EL" in full_df.columns and pd.notna(row.get("EL", None)):
        risk_block["EL"] = float(row["EL"])
    if "EL_change_vs_baseline_pct" in full_df.columns and pd.notna(row.get("EL_change_vs_baseline_pct", None)):
        risk_block["EL_change_vs_baseline_pct"] = float(row["EL_change_vs_baseline_pct"])

    # ---- macro block ----
    macro_block = {}
    for c in macro_cols:
        val = row.get(c, None)
        if pd.notna(val):
            # convert numpy -> Python scalar
            try:
                macro_block[c] = float(val)
            except Exception:
                macro_block[c] = val

    record = {
        "scenario": scen_name,
        "family": family,
        "macro": macro_block,
        "risk": risk_block,
    }
    scenario_records.append(record)


In [43]:
scenario_records

[{'scenario': 'baseline_actual',
  'family': 'baseline',
  'macro': {},
  'risk': {'mean_pd': 0.2048534991205811,
   'p50_pd': 0.1752750604390484,
   'p90_pd': 0.3914056910148148,
   'p99_pd': 0.6176613576868888,
   'uplift_vs_baseline_pct': 0.0,
   'EL': 729539408.2187891,
   'EL_change_vs_baseline_pct': 0.0}},
 {'scenario': 'mild_adverse',
  'family': 'data_driven',
  'macro': {'GDPC1': 18782.243,
   'UNRATE': 5.433333333333334,
   'CPIAUCSL': 247.23833333333337,
   'FEDFUNDS': 1.2033333333333334,
   'UNRATE_delta_qoq': -0.0130718954248367,
   'FEDFUNDS_delta_qoq': 0.2022160664819943,
   'GDPC1_delta_qoq': 0.0040024506125282,
   'inflation_qoq': 0.0063478253641482,
   'real_rate_qoq': 0.1541756763574446},
  'risk': {'mean_pd': 0.2634378118933922,
   'p50_pd': 0.2533744418557493,
   'p90_pd': 0.4177165679179622,
   'p99_pd': 0.5660747089032947,
   'uplift_vs_baseline_pct': 28.59815088553952,
   'EL': 938174188.9506706,
   'EL_change_vs_baseline_pct': 28.598150885539543}},
 {'scenario'

In [44]:

# 4) Save JSON + flat CSV
import json
json_path = RES_DIR / "llm_scenarios.json"
flat_csv_path = RES_DIR / "llm_scenarios_flat.csv"

with json_path.open("w", encoding="utf-8") as f:
    json.dump(scenario_records, f, ensure_ascii=False, indent=2)

# Flatten for CSV inspection
flat_rows = []
for rec in scenario_records:
    base = {
        "scenario": rec["scenario"],
        "family": rec["family"],
        "mean_pd": rec["risk"].get("mean_pd"),
        "EL": rec["risk"].get("EL"),
        "EL_change_vs_baseline_pct": rec["risk"].get("EL_change_vs_baseline_pct"),
        "p50_pd": rec["risk"].get("p50_pd"),
        "p90_pd": rec["risk"].get("p90_pd"),
        "p99_pd": rec["risk"].get("p99_pd"),
        "uplift_vs_baseline_pct": rec["risk"].get("uplift_vs_baseline_pct"),
    }
    # Add macro_* columns
    for k, v in rec["macro"].items():
        base[f"macro_{k}"] = v
    flat_rows.append(base)

flat_df = pd.DataFrame(flat_rows)
flat_df.to_csv(flat_csv_path, index=False)

print(f"   Prepared {len(scenario_records)} scenarios for LLM.")


   Prepared 16 scenarios for LLM.


In [45]:
flat_df

,scenario,family,mean_pd,EL,EL_change_vs_baseline_pct,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,macro_GDPC1,macro_UNRATE,macro_CPIAUCSL,macro_FEDFUNDS,macro_UNRATE_delta_qoq,macro_FEDFUNDS_delta_qoq,macro_GDPC1_delta_qoq,macro_inflation_qoq,macro_real_rate_qoq,macro_CPIAUCSL_delta_qoq
0,baseline_actual,baseline,0.204853,7.295394e+08,0.000000,0.175275,0.391406,0.617661,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mild_adverse,data_driven,0.263438,9.381742e+08,28.598151,0.253374,0.417717,0.566075,28.598151,18782.243000,5.433333,247.238333,1.203333,-0.013072,0.202216,0.004002,0.006348,0.154176,NaN
2,severe_adverse,data_driven,0.151683,5.401834e+08,-25.955562,0.139560,0.255326,0.384049,-25.955562,17953.974000,6.933333,251.686333,1.923333,0.006803,0.555556,0.001845,0.007955,0.190550,NaN
3,Fed_Adverse,Fed,0.022956,8.175096e+07,-88.794167,0.019596,0.040539,0.071352,-88.794167,18821.441681,5.475000,257.030458,0.225000,0.800000,-0.500000,-0.035000,0.013000,-0.513000,NaN
4,Fed_Baseline,Fed,0.242373,8.631555e+08,18.315129,0.231391,0.388901,0.536454,18.315129,21214.070968,3.925000,260.107147,1.775000,0.000000,0.300000,0.028000,0.019000,0.281000,NaN
5,Fed_Severe,Fed,0.240374,8.560379e+08,17.339507,0.229316,0.386124,0.533543,17.339507,16907.187783,6.900000,254.741601,0.100000,1.500000,0.000000,-0.089000,0.009000,-0.009000,NaN
6,GenAI_S1,GenAI,0.248111,8.835923e+08,21.116461,0.237358,0.396833,0.544713,21.116461,18085.832031,6.714330,232.857193,0.384288,-0.020352,-0.047811,0.002063,0.004348,-0.052159,0.004348
7,GenAI_S2,GenAI,0.317561,1.130920e+09,55.018363,0.310747,0.487981,0.634117,55.018363,18106.210938,6.799571,232.539124,0.416461,-0.008615,0.030968,0.005793,0.004808,0.026160,0.004808
8,GenAI_S3,GenAI,0.116683,4.155417e+08,-43.040544,0.105570,0.199684,0.312113,-43.040544,18098.371094,6.770442,231.740982,0.307540,0.000444,-0.265904,0.006234,-0.000793,-0.265110,-0.000793
9,GenAI_S4,GenAI,0.299943,1.068179e+09,46.418297,0.291945,0.465700,0.613154,46.418297,18077.287109,6.811875,232.070328,0.407448,-0.005914,0.007856,0.004799,0.003381,0.004476,0.003381


In [55]:
# === 0) Imports & setup ==========================================
from pathlib import Path
import json

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

# Assume flat_df already exists from your earlier steps
# flat_df columns (example):
# ['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct',
#  'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct',
#  'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', ... ]

RES_DIR = Path("../results")
llm_path = RES_DIR / "llm_narratives.json"

# Work on a copy so we don't mutate flat_df accidentally
merged_df = flat_df.copy()

print("Initial merged_df columns:")
print(merged_df.columns.tolist())


Initial merged_df columns:
['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct', 'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', 'macro_UNRATE_delta_qoq', 'macro_FEDFUNDS_delta_qoq', 'macro_GDPC1_delta_qoq', 'macro_inflation_qoq', 'macro_real_rate_qoq', 'macro_CPIAUCSL_delta_qoq']


In [56]:
# === 1) Load LLM narratives JSON and merge =======================

if llm_path.exists():
    with open(llm_path, "r") as f:
        llm_data = json.load(f)

    llm_df = pd.DataFrame(llm_data)

    # Align columns with our scenario table
    # JSON example:
    # {
    #   "scenario_name": "...",
    #   "headline": "...",
    #   "macro_story": "...",
    #   "credit_risk_impact": "...",
    #   "comparison_to_baseline": "...",
    #   "key_risks": [...],
    #   "management_actions": [...],
    #   "tone": "benign",
    #   "scenario_family": "baseline"
    # }
    llm_df = llm_df.rename(
        columns={
            "scenario_name": "scenario",
            "scenario_family": "llm_family"
        }
    )

    # Keep only the narrative-related columns
    narrative_cols = [
        "scenario",
        "headline",
        "macro_story",
        "credit_risk_impact",
        "comparison_to_baseline",
        "key_risks",
        "management_actions",
        "tone",
        "llm_family",
    ]
    # Some fields may not exist depending on how you generated JSON
    narrative_cols = [c for c in narrative_cols if c in llm_df.columns]

    merged_df = merged_df.merge(
        llm_df[narrative_cols],
        on="scenario",
        how="left",
        validate="one_to_one"
    )

    print("LLM narratives merged. Columns now:")
    print(merged_df.columns.tolist())
else:
    print(f"LLM narrative file not found at: {llm_path}")


LLM narratives merged. Columns now:
['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct', 'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', 'macro_UNRATE_delta_qoq', 'macro_FEDFUNDS_delta_qoq', 'macro_GDPC1_delta_qoq', 'macro_inflation_qoq', 'macro_real_rate_qoq', 'macro_CPIAUCSL_delta_qoq', 'headline', 'macro_story', 'credit_risk_impact', 'comparison_to_baseline', 'key_risks', 'management_actions', 'tone', 'llm_family']


In [59]:
# === 2) Helper: pretty printer for a single scenario =============

def show_scenario(name: str):
    """Render one scenario (macro + risk + LLM narrative) as Markdown."""
    r = merged_df.loc[merged_df["scenario"] == name]
    if r.empty:
        print(f"Scenario '{name}' not found in merged_df.")
        return
    r = r.iloc[0]

    md = f"## Scenario: **{r['scenario']}**  \n"
    md += f"Family: **{r['family']}**  \n\n"

    # --- Macro block ------------------------------------------------
    macro_cols = [c for c in merged_df.columns if c.startswith("macro_")]
    if macro_cols:
        md += "### Macro Environment\n"
        for col in macro_cols:
            val = r.get(col, None)
            if pd.notna(val):
                raw_name = col.replace("macro_", "")
                if isinstance(val, (int, float, np.number)):
                    md += f"- **{raw_name}**: {float(val):,.4f}\n"
                else:
                    md += f"- **{raw_name}**: {val}\n"
        md += "\n"

    # --- Risk metrics block -----------------------------------------
    md += "### Risk Metrics\n"
    risk_cols = [
        "mean_pd",
        "p50_pd", "p90_pd", "p99_pd",
        "EL", "EL_change_vs_baseline_pct",
        "uplift_vs_baseline_pct",
    ]
    for col in risk_cols:
        if col in merged_df.columns and pd.notna(r.get(col, None)):
            val = r[col]
            if isinstance(val, (int, float, np.number)):
                md += f"- **{col}**: {float(val):,.4f}\n"
            else:
                md += f"- **{col}**: {val}\n"
    md += "\n"

    # --- LLM narrative block ----------------------------------------
    if "headline" in merged_df.columns and pd.notna(r.get("headline", None)):
        md += "### LLM Narrative\n"
        md += f"**Headline:** {r['headline']}\n\n"

        if pd.notna(r.get("macro_story", None)):
            md += f"**Macro story.** {r['macro_story']}\n\n"

        if pd.notna(r.get("credit_risk_impact", None)):
            md += f"**Credit risk impact.** {r['credit_risk_impact']}\n\n"

        if pd.notna(r.get("comparison_to_baseline", None)):
            md += f"**Comparison to baseline.** {r['comparison_to_baseline']}\n\n"

        # key_risks and management_actions might be list *or* string
        kr = r.get("key_risks", None)
        if isinstance(kr, list):
            md += "**Key risks:**\n"
            for k in kr:
                md += f"- {k}\n"
            md += "\n"
        elif isinstance(kr, str) and kr.strip():
            md += f"**Key risks:** {kr}\n\n"

        ma = r.get("management_actions", None)
        if isinstance(ma, list):
            md += "**Suggested management actions:**\n"
            for a in ma:
                md += f"- {a}\n"
            md += "\n"
        elif isinstance(ma, str) and ma.strip():
            md += f"**Suggested management actions:** {ma}\n\n"

        if pd.notna(r.get("tone", None)):
            md += f"_Narrative tone: **{r['tone']}**._\n"
    else:
        md += "### LLM Narrative\n"
        md += "*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*\n"

    display(Markdown(md))


In [60]:
# === 5) Show examples ===
show_scenario("baseline_actual")


## Scenario: **baseline_actual**  
Family: **baseline**  

### Macro Environment
- **story**: The macroeconomic environment in this scenario is assumed to be stable and steady, with moderate economic growth and a resilient labor market supporting sustained employment levels. Inflation remains within target ranges, allowing for stable interest rates without significant volatility. This scenario reflects an absence of major shocks to GDP, labor markets, inflation, or monetary policy, representing typical or expected economic conditions.

### Risk Metrics
- **mean_pd**: 0.2049
- **p50_pd**: 0.1753
- **p90_pd**: 0.3914
- **p99_pd**: 0.6177
- **EL**: 729,539,408.2188
- **EL_change_vs_baseline_pct**: 0.0000
- **uplift_vs_baseline_pct**: 0.0000

### LLM Narrative
**Headline:** The baseline_actual scenario reflects a stable macroeconomic environment with moderate credit risk consistent with average expectations.

**Macro story.** The macroeconomic environment in this scenario is assumed to be stable and steady, with moderate economic growth and a resilient labor market supporting sustained employment levels. Inflation remains within target ranges, allowing for stable interest rates without significant volatility. This scenario reflects an absence of major shocks to GDP, labor markets, inflation, or monetary policy, representing typical or expected economic conditions.

**Credit risk impact.** Given the stable macroeconomic environment, credit risk metrics remain consistent with normal conditions. The mean probability of default is around 20%, with median and tail PDs reflecting average risk levels. Expected losses are stable and do not show any increase compared to baseline. The lack of deterioration in key macro variables translates to no significant upward pressure on default probabilities or credit losses.

**Comparison to baseline.** Since no explicit baseline was identified, this scenario effectively serves as the reference point for credit risk assessment. There are no changes in PD or expected loss relative to baseline, highlighting that this scenario captures the bank’s normal operating environment without added stress. It can be regarded as the standard benchmark against which more adverse conditions would be measured.

**Key risks:**
- Sustained moderate economic growth maintaining loan performance
- Stable labor market with controlled unemployment rates
- Inflation and interest rates remaining within expected targets

**Suggested management actions:**
- Maintain current capital and provisioning policies aligned with baseline credit risk
- Continue regular monitoring of economic indicators to identify early signs of deterioration
- Focus on risk pricing and limit setting consistent with stable credit risk environment

_Narrative tone: **benign**._


In [61]:
show_scenario("mild_adverse")

## Scenario: **mild_adverse**  
Family: **data_driven**  

### Macro Environment
- **GDPC1**: 18,782.2430
- **UNRATE**: 5.4333
- **CPIAUCSL**: 247.2383
- **FEDFUNDS**: 1.2033
- **UNRATE_delta_qoq**: -0.0131
- **FEDFUNDS_delta_qoq**: 0.2022
- **GDPC1_delta_qoq**: 0.0040
- **inflation_qoq**: 0.0063
- **real_rate_qoq**: 0.1542
- **story**: The macroeconomic environment shows modest real GDP growth with a quarterly increase of 0.4%, indicating a moderate expansion. The labor market improves slightly as unemployment falls by 0.0131 percentage points to 5.4333%. Inflation remains controlled, with a quarterly CPI increase of 0.63%. The policy interest rate rises moderately by 0.2022 points to 1.2033%, while the real interest rate is positive but low at 0.1542%. Overall, the environment reflects mild economic stress with limited inflationary pressure and gradual tightening of monetary policy.

### Risk Metrics
- **mean_pd**: 0.2634
- **p50_pd**: 0.2534
- **p90_pd**: 0.4177
- **p99_pd**: 0.5661
- **EL**: 938,174,188.9507
- **EL_change_vs_baseline_pct**: 28.5982
- **uplift_vs_baseline_pct**: 28.5982

### LLM Narrative
**Headline:** Mild adverse scenario characterized by moderate economic expansion and improving labor market with heightened credit risk.

**Macro story.** The macroeconomic environment shows modest real GDP growth with a quarterly increase of 0.4%, indicating a moderate expansion. The labor market improves slightly as unemployment falls by 0.0131 percentage points to 5.4333%. Inflation remains controlled, with a quarterly CPI increase of 0.63%. The policy interest rate rises moderately by 0.2022 points to 1.2033%, while the real interest rate is positive but low at 0.1542%. Overall, the environment reflects mild economic stress with limited inflationary pressure and gradual tightening of monetary policy.

**Credit risk impact.** The mild adverse conditions translate into a noticeable increase in credit risk. The mean probability of default rises by approximately 28.6%, from 20.5% at baseline to 26.3%, indicating wider borrower stress. Expected losses also increase by a similar proportion, reflecting higher anticipated credit losses. Median and tail PDs exhibit elevated values, though extreme tail risk (p99) slightly declines compared with baseline, pointing to moderate deterioration rather than severe distress. The high unemployment rate combined with moderate policy rate increases are the main contributors elevating borrower risk.

**Comparison to baseline.** Compared with the baseline scenario, the mild adverse environment produces a clear but moderate increase in credit risk metrics and expected losses. While PDs and losses increase by nearly 29%, the overall stress remains contained without extreme spikes in tail risk. This represents a deterioration from baseline but does not approach a severe stress state, supporting characterization as ‘mild adverse’. The scenario highlights vulnerabilities in borrower capacity amid tightening rates without outright economic contraction.

**Key risks:**
- Moderate increases in policy interest rates contributing to higher credit costs
- Persistently elevated yet slightly declining unemployment rate
- Sustained inflationary pressures at moderate levels
- Moderate real GDP growth insufficient to fully offset labor and credit stress

**Suggested management actions:**
- Review capital adequacy to ensure sufficient buffers for increased expected losses
- Strengthen credit underwriting standards with focus on rate-sensitive borrowers
- Enhance monitoring of loan portfolios vulnerable to interest rate and labor market shifts

_Narrative tone: **cautious**._


In [62]:
show_scenario("Fed_Severe")

## Scenario: **Fed_Severe**  
Family: **Fed**  

### Macro Environment
- **GDPC1**: 16,907.1878
- **UNRATE**: 6.9000
- **CPIAUCSL**: 254.7416
- **FEDFUNDS**: 0.1000
- **UNRATE_delta_qoq**: 1.5000
- **FEDFUNDS_delta_qoq**: 0.0000
- **GDPC1_delta_qoq**: -0.0890
- **inflation_qoq**: 0.0090
- **real_rate_qoq**: -0.0090
- **story**: The macroeconomic environment shows a significant contraction in real GDP with a quarterly decline of 8.9%. The labor market weakens substantially, as the unemployment rate rises to 6.9%, an increase of 1.5 percentage points quarterly. Inflation remains modest with a 0.9% quarterly increase in the CPI index, while nominal policy rates remain near zero, and real interest rates stay slightly negative. This environment suggests a severe recessionary phase with limited monetary policy adjustment capacity.

### Risk Metrics
- **mean_pd**: 0.2404
- **p50_pd**: 0.2293
- **p90_pd**: 0.3861
- **p99_pd**: 0.5335
- **EL**: 856,037,943.3880
- **EL_change_vs_baseline_pct**: 17.3395
- **uplift_vs_baseline_pct**: 17.3395

### LLM Narrative
**Headline:** The Fed_Severe scenario depicts a sharp economic downturn with rising unemployment, low growth, subdued inflation, and minimal interest rate movement, resulting in elevated credit risk.

**Macro story.** The macroeconomic environment shows a significant contraction in real GDP with a quarterly decline of 8.9%. The labor market weakens substantially, as the unemployment rate rises to 6.9%, an increase of 1.5 percentage points quarterly. Inflation remains modest with a 0.9% quarterly increase in the CPI index, while nominal policy rates remain near zero, and real interest rates stay slightly negative. This environment suggests a severe recessionary phase with limited monetary policy adjustment capacity.

**Credit risk impact.** The economic stress is reflected in higher credit risk metrics, with mean probability of default increasing by 17.3% relative to baseline, reaching 24%. Expected losses also rise by the same magnitude, indicating a material increase in portfolio credit deterioration. Despite some tail PD measures showing less increase, the overall expected loss rise highlights broad credit quality weakening driven by worsening labor market conditions and economic contraction.

**Comparison to baseline.** Compared to the baseline_actual scenario, the Fed_Severe scenario presents a marked deterioration in credit risk indicators with a sizable rise in mean PD and expected loss. The unemployment-driven increase in defaults outweighs the marginal change in inflation and interest rates, which remain subdued. This scenario is materially more adverse than baseline, reflecting a deeper downturn and more stressed credit environment.

**Key risks:**
- Sharp GDP contraction signaling broad economic weakness
- Significant rise in unemployment rate, increasing default likelihood
- Minimal policy rate movement limits monetary easing options
- Sustained but moderate inflation, insufficient to offset real rate pressures

**Suggested management actions:**
- Increase capital buffers to absorb higher expected credit losses
- Tighten credit underwriting criteria and portfolio concentration limits
- Enhance monitoring of borrower credit quality and early warning indicators

_Narrative tone: **severe**._


In [63]:
show_scenario("GenAI_S7")

## Scenario: **GenAI_S7**  
Family: **GenAI**  

### Macro Environment
- **GDPC1**: 18,218.5527
- **UNRATE**: 6.8315
- **CPIAUCSL**: 233.2491
- **FEDFUNDS**: 0.5457
- **UNRATE_delta_qoq**: -0.0096
- **FEDFUNDS_delta_qoq**: 0.4701
- **GDPC1_delta_qoq**: 0.0127
- **inflation_qoq**: 0.0085
- **real_rate_qoq**: 0.4616
- **CPIAUCSL_delta_qoq**: 0.0085
- **story**: The macroeconomic environment in the GenAI_S7 scenario shows modest real GDP growth of 1.27% quarterly, indicating slow but positive expansion. The labor market is relatively weak, with an unemployment rate elevated at 6.83%, though improving slightly over the quarter. Inflation remains persistent, with the CPI rising by 0.85% quarterly, sustaining inflationary pressure. Monetary policy is tightening sharply as reflected by a policy rate rising by 47 basis points to 0.55%, pushing real interest rates higher to approximately 0.46%.

### Risk Metrics
- **mean_pd**: 0.7465
- **p50_pd**: 0.7708
- **p90_pd**: 0.8767
- **p99_pd**: 0.9282
- **EL**: 2,658,332,889.8146
- **EL_change_vs_baseline_pct**: 264.3851
- **uplift_vs_baseline_pct**: 264.3851

### LLM Narrative
**Headline:** GenAI_S7 scenario depicts an adverse economic environment with elevated credit risk driven by rapid interest rate increases and persistent inflation pressure.

**Macro story.** The macroeconomic environment in the GenAI_S7 scenario shows modest real GDP growth of 1.27% quarterly, indicating slow but positive expansion. The labor market is relatively weak, with an unemployment rate elevated at 6.83%, though improving slightly over the quarter. Inflation remains persistent, with the CPI rising by 0.85% quarterly, sustaining inflationary pressure. Monetary policy is tightening sharply as reflected by a policy rate rising by 47 basis points to 0.55%, pushing real interest rates higher to approximately 0.46%.

**Credit risk impact.** This tightening cycle combined with elevated inflation and a still fragile labor market translates into significantly increased credit risk. The mean probability of default surges to 74.6%, a more than threefold increase compared to baseline. Both median and tail risk metrics similarly rise, reflecting broader borrower distress. Expected losses escalate substantially to approximately 2.66 billion currency units, underscoring severe portfolio stress. The combination of rising rates and persistent inflation appears to be the dominant credit risk drivers, increasing repayment burdens and reducing economic resilience.

**Comparison to baseline.** Compared to the baseline scenario, GenAI_S7 exhibits markedly higher credit risk, with mean PD rising by over 260% and expected losses rising by the same magnitude. The baseline reflects a low rate, moderate default environment, while GenAI_S7 stresses key vulnerabilities via sustained inflation and rapid policy tightening. This represents a material adverse shift requiring close attention and proactive risk mitigation.

**Key risks:**
- Rapid increase in policy interest rates raising borrower funding costs
- Persistently elevated inflation sustaining economic uncertainty
- Elevated unemployment rate, despite slight improvement, limiting income stability

**Suggested management actions:**
- Review and potentially increase capital buffers to absorb higher expected losses
- Tighten credit underwriting and pricing to reflect increased risks and mitigate further exposures
- Enhance portfolio monitoring for early signs of borrower stress, especially in vulnerable sectors

_Narrative tone: **adverse**._


In [64]:
show_scenario("GenAI_S8")

## Scenario: **GenAI_S8**  
Family: **GenAI**  

### Macro Environment
- **GDPC1**: 18,082.7383
- **UNRATE**: 6.9070
- **CPIAUCSL**: 231.9786
- **FEDFUNDS**: 0.4418
- **UNRATE_delta_qoq**: 0.0120
- **FEDFUNDS_delta_qoq**: 0.0201
- **GDPC1_delta_qoq**: 0.0064
- **inflation_qoq**: 0.0027
- **real_rate_qoq**: 0.0174
- **CPIAUCSL_delta_qoq**: 0.0027
- **story**: The GenAI_S8 scenario presents a moderate but positive economic growth environment, with real GDP increasing slightly by 0.64% quarterly. However, the labor market weakens as the unemployment rate rises to nearly 7%, indicating emerging job market stress. Inflation remains low but positive, with a quarterly CPI inflation rate of 0.27%, while policy rates are gradually increasing, reaching 0.44%. Real interest rates remain slightly positive, reflecting a cautious monetary policy stance.

### Risk Metrics
- **mean_pd**: 0.3051
- **p50_pd**: 0.2975
- **p90_pd**: 0.4723
- **p99_pd**: 0.6194
- **EL**: 1,086,613,102.0884
- **EL_change_vs_baseline_pct**: 48.9451
- **uplift_vs_baseline_pct**: 48.9451

### LLM Narrative
**Headline:** Moderate economic growth with rising unemployment and inflation results in elevated credit risk and expected losses compared to baseline.

**Macro story.** The GenAI_S8 scenario presents a moderate but positive economic growth environment, with real GDP increasing slightly by 0.64% quarterly. However, the labor market weakens as the unemployment rate rises to nearly 7%, indicating emerging job market stress. Inflation remains low but positive, with a quarterly CPI inflation rate of 0.27%, while policy rates are gradually increasing, reaching 0.44%. Real interest rates remain slightly positive, reflecting a cautious monetary policy stance.

**Credit risk impact.** This environment of slowing labor market conditions alongside modest growth translates into higher credit risk metrics. Mean probability of default (PD) increases by nearly 49% compared to the baseline, pushing expected losses up by a similar magnitude to over 1 billion currency units. Median and tail PDs also show marked increases, signaling broader portfolio vulnerability. The upward credit risk is driven by weakening borrower capacity amid rising unemployment and incremental rate pressures, which exacerbate default likelihood and loss severity.

**Comparison to baseline.** Compared to the baseline scenario, which assumes more stable conditions, this stress scenario exhibits a significant deterioration in credit quality and loss expectations. The mean PD rises from approximately 20.5% baseline to 30.5%, and expected losses jump correspondingly. While inflation and rates move modestly, the dominant difference is the elevated unemployment rate driving the material increase in default risk.

**Key risks:**
- Rising unemployment exerting pressure on borrower repayment capacity
- Moderate but insufficient GDP growth to offset labor market weakness
- Gradual policy rate increases raising funding costs and borrower stress

**Suggested management actions:**
- Enhance credit monitoring and early warning systems focusing on sectors sensitive to unemployment
- Review capital buffers to ensure adequacy under higher expected losses
- Consider tightening underwriting standards and revising pricing to reflect increased risk

_Narrative tone: **adverse**._
